In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import re
import requests
import math
import time
import numpy as np
from tqdm import tqdm
import os

In [2]:
BASE_URL = "https://www.kjnala.com"
search_url = f'{BASE_URL}/board/index.jsp?code=bbs018&page=1'
page = requests.get(search_url)
soup = BeautifulSoup(page.content, 'html.parser')

num = soup.select('td.pc_line')[0].text
end_num = math.ceil(int(num) / 10)

df = pd.read_csv('통합.csv', encoding = 'cp949')
stop_date = df['date'].iloc[0]
print(stop_date)

2024-03-15


In [3]:
# Login credentials
username = os.getenv('id1')
password = os.getenv('pswd1')

# Login endpoint
login_url = 'https://www.kjnala.com/member/login.jsp'

# Page to scrape after login
scrape_url = 'https://www.kjnala.com/board/read.jsp?id=1852426&code=bbs018&page=1'

# Create a session
session = requests.Session()

# Perform login
login_data = {
    'id': username,
    'passwd': password
}
response = session.post(login_url, data=login_data)
print(response.text)

<script>parent.location.replace('/mypage/index.jsp');</script>


In [4]:
def get_items(date, dtype, link):
    search_url = f'{BASE_URL}{link}'
    scrape_response = session.get(search_url)
    soup = BeautifulSoup(scrape_response.text, 'html.parser')

    items = soup.select('div.contArea > p')
    
    # Filter items with more than 9 characters
    filtered_items = [item for item in items if len(item.text) >= 9]

    # Reassign items with filtered items
    items = filtered_items
    
    count = str(items).count('<p>')
    
    item_list = []
    
    for i in range(len(items)):
        if '<br/>' in str(items[i]) or '<br>' in str(items[i]):
            item_t = re.sub('<p>', '', str(items[i]))
            item_t = re.sub('</p>', '', item_t)
            item_t = item_t.replace(u'\xa0', u'')
            item_t = re.split(r'<br/>|<br>', item_t)
            while("" in item_t):
                item_t.remove("")

            item_t =[s[1:].lstrip() if s.startswith('-') else s.lstrip('- ').lstrip() for s in item_t]
            
            if count > 3:
                item_t = [''.join(item_t)]
            
            for i in range(len(item_t)):
                if '-' in item_t[i]:
                    item = item_t[i].split('-', 1)[0]
                    item2 = item_t[i].split('-', 1)[1]
                    item_list.append([date, item.rstrip(), item2.lstrip(), dtype])
                elif ' ' in item_t[i]:
                    item = item_t[i].split(' ', 1)[0]
                    item2 = item_t[i].split(' ', 1)[1]
                    item_list.append([date, item.rstrip(), item2.lstrip(), dtype])
                else: 
                    item_list.append([date, item_t[i], np.nan, dtype])

    if item_list == []:
        for i in range(len(items)):
            item_t = re.sub('<p>', '', str(items[i]))
            item_t = re.sub('</p>', '', item_t)
            item_t = item_t.replace(u'\xa0', u'')
                    
            if '-' in item_t:
                item = item_t.split('-', 1)[0]
                item2 = item_t.split('-', 1)[1]
                item_list.append([date, item.rstrip(), item2.lstrip(), dtype])
            elif ' ' in item_t:
                item = item_t.split(' ', 1)[0]
                item2 = item_t.split(' ', 1)[1]
                item_list.append([date, item.rstrip(), item2.lstrip(), dtype])
            else: 
                item_list.append([date, item_t[i], np.nan, dtype])
    
    return item_list

In [5]:
data = []
def get_link(page):
    search_url = f'{BASE_URL}/board/index.jsp?code=bbs018&page={page}'
    page = requests.get(search_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    global stop_reached  # Use global flag
    
    nname = soup.select('p.subject > a')
    
    for i in range(len(nname)):
        title = re.sub('\s+', '', nname[i].text)
        if '사위' in title:
            dtype = '사위일체'
        elif '효자' in title:
            dtype = '효자종목'
        elif '저점' in title:
            dtype = '저점장대양봉'
        else:
            continue
        number = re.findall("\d+", title)
        date = '-'.join(number)[0:10]
        
        if date == stop_date:
            stop_reached = True  # Set the flag to indicate stop_date is reached
            return data  # Exit the function
        
        # Skip get_items if the length of the date string is less than 5
        if len(date) < 5:
            continue
        
        link = nname[i]['href']
        
        try: 
            data.extend(get_items(date, dtype, link))
            
        except:
            print(date)
    return data

In [6]:
stop_reached = False  # Flag to track stop_date condition
for i in tqdm(range(1, end_num+1)): 
    data = get_link(i)
    if stop_reached:  # Check if stop_date is reached inside get_link
        break  # Break out of the loop if the stop_date is reached
    time.sleep(2)

  0%|          | 0/70 [00:00<?, ?it/s]

  9%|▊         | 6/70 [00:47<08:31,  7.99s/it]


In [7]:
df = pd.DataFrame(data, columns = ['date', 'item', 'exp', 'type'])
df['integrated_category'] = '' 
df['item'] = df['item'].str.replace('amp;', '')
df['exp'] = df['exp'].str.replace('exp;', '')
df['exp'] =df['exp'].str.replace('</br>', '')
df = df[~df.item.str.contains('=', na=False)]
df = df[~df.exp.str.contains('=', na=False)]

length_condition = df['item'].apply(lambda x: len(str(x))) >= 2
df = df[length_condition]

# 설명 없는 종목 제거
df = df.dropna(subset=['exp'])

# '금일 사위일체는 없습니다' 제거
condition = df['item'].str.contains('사위일체') | df['exp'].str.contains('사위일체')
df = df[~condition]

# 종목 이름에 '효자'있는 종목 제거
df = df[~df['item'].str.contains('효자')]


df.reset_index(drop=True, inplace=True)

In [8]:
nt_word = ['잘못', '큰일', '버리자', '오케이', '애매', '멀리', '피한', '리스크', '걸린다', '피하', '본척', '피해', '아슬', '우려', '못쓰고', '더 좋은', '다음', '즐비', '위험', 
           '말함', '앉음', '꺼려', '비리비리', '내리막', '무섭', '무조건', '안된다', '뒷설거지', '거둬야', '많은분', '어렵', '스탑', '두렵', '다소', '경계', '허봉', '멀다', 
           '흘러', '무너질지', '부담', '갈수록', '피해야', '비교', '종목에 대한 사랑', '톱니바퀴', '피함', '못넘고', '벽타기', '못먹어도', '불안', '아님', '목장', '기다려야', 
           '밀리', '털었다', '흉측', '버려', '붕괴', '결투']
df = df[~df.exp.str.contains('|'.join(nt_word))]
df = df.reset_index(drop=True)
len(df.index)

636

In [9]:
df

,date,item,exp,type,integrated_category
0,2024-04-09,스마트레이더시스템,좀 더 지켜봐야한다,저점장대양봉,
1,2024-04-09,HD현대에너지솔루션,월봉을 보면 좀 더 내려오는게 안전하지 않나,저점장대양봉,
2,2024-04-09,나이벡,이만원 넘으면 다중봉 다중턱,저점장대양봉,
3,2024-04-09,한화투자증권,앞폭탄 맞고 떨어지는 상황,저점장대양봉,
4,2024-04-09,세기상사,상황을 지켜보는 것도 하나의 방법,저점장대양봉,
...,...,...,...,...,...
631,2024-03-18,동구바이오제약,앞으로 조금 더 가지 않을까,효자종목,
632,2024-03-18,파미셀,이 종목도 더욱더 우리한테 기회를 줄 것 같다,효자종목,
633,2024-03-18,솔트웨어,반갑기 그지 없다,효자종목,
634,2024-03-18,디와이디,지금 사면 더 싸게 사는 상황이 아닌가,사위일체,


In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow as tf
import random

# Set seeds to ensure reproducibility
tf.random.set_seed(2)  # TensorFlow random seed
np.random.seed(2)     # Numpy random seed
random.seed(2)        # Python random seed

# Load data
data = pd.read_csv('통합.csv', encoding = 'cp949')# Replace with your data file

# Split data into training and testing sets
X = data['exp']
y = data['integrated_category']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Tokenize text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)

max_words = len(tokenizer.index_word) + 1
# Pad sequences to ensure uniform length
max_seq_length = max(max(len(seq) for seq in X_train_seq), max(len(seq) for seq in X_val_seq)) # Change as needed based on your data
X_train_pad = pad_sequences(X_train_seq, maxlen=max_seq_length)
X_val_pad = pad_sequences(X_val_seq, maxlen=max_seq_length)

# Encode labels
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_val = label_encoder.transform(y_val)

# Build CNN model
""" model_cnn = Sequential()
model_cnn.add(Embedding(max_words, 32, input_length=max_seq_length))
model_cnn.add(Conv1D(256, 10, activation='relu'))
model_cnn.add(GlobalMaxPooling1D())
model_cnn.add(Dense(144, activation='relu'))
model_cnn.add(Dropout(0.4))
model_cnn.add(Dense(96, activation='relu'))
model_cnn.add(Dropout(0.2)) """

model_cnn = Sequential()
model_cnn.add(Embedding(max_words, 32, input_length=max_seq_length))
model_cnn.add(Conv1D(128, 5, activation='relu'))
model_cnn.add(GlobalMaxPooling1D())
model_cnn.add(Dense(64, activation='relu'))
model_cnn.add(Dropout(0.4))
model_cnn.add(Dense(len(label_encoder.classes_), activation='softmax'))


# Compile and train the CNN model
model_cnn.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

# Train the CNN model with callbacks
model_cnn.fit(X_train_pad, y_train, epochs=100, batch_size=256, validation_data = (X_val_pad, y_val), callbacks=[early_stopping, model_checkpoint])

# Evaluate the CNN model
y_pred_proba_cnn = model_cnn.predict(X_val_pad)  # Get predicted probabilities for each class
y_pred_cnn = y_pred_proba_cnn.argmax(axis=1)  # Extract classes based on highest probability

accuracy_cnn = accuracy_score(y_val, y_pred_cnn)
classification_rep_cnn = classification_report(y_val, y_pred_cnn)

print(f"Accuracy for CNN: {accuracy_cnn}")
print(f"Classification Report for CNN:\n{classification_rep_cnn}")

# Assuming df2 has a column 'exp' containing text data

# Tokenize text data in df2
X_new = df['exp']
X_new_seq = tokenizer.texts_to_sequences(X_new)
X_new_pad = pad_sequences(X_new_seq, maxlen=max_seq_length)

# Predict category values using the trained CNN model
y_pred_proba_new = model_cnn.predict(X_new_pad)
y_pred_new = y_pred_proba_new.argmax(axis=1)

# Convert predicted labels back to category names (if needed)
predicted_categories = label_encoder.inverse_transform(y_pred_new)

# Add predicted values to a new column in df2
df['integrated_category'] = predicted_categories

Epoch 1/100
20/20 [==============================] - 3s 39ms/step - loss: 3.7412 - accuracy: 0.1792 - val_loss: 3.4301 - val_accuracy: 0.1992
Epoch 2/100
 7/20 [=========>....................] - ETA: 0s - loss: 3.4632 - accuracy: 0.1797

c:\Users\Administrator\anaconda3\envs\stock\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


20/20 [==============================] - 0s 25ms/step - loss: 3.4154 - accuracy: 0.1782 - val_loss: 3.2805 - val_accuracy: 0.1992
Epoch 3/100
20/20 [==============================] - 1s 26ms/step - loss: 3.2929 - accuracy: 0.1992 - val_loss: 3.1448 - val_accuracy: 0.1992
Epoch 4/100
20/20 [==============================] - 1s 29ms/step - loss: 3.0176 - accuracy: 0.2308 - val_loss: 2.8337 - val_accuracy: 0.2742
Epoch 5/100
20/20 [==============================] - 1s 27ms/step - loss: 2.6535 - accuracy: 0.2988 - val_loss: 2.5770 - val_accuracy: 0.3427
Epoch 6/100
20/20 [==============================] - 0s 25ms/step - loss: 2.3412 - accuracy: 0.3836 - val_loss: 2.3732 - val_accuracy: 0.4008
Epoch 7/100
20/20 [==============================] - 1s 29ms/step - loss: 2.0860 - accuracy: 0.4421 - val_loss: 2.2450 - val_accuracy: 0.4379
Epoch 8/100
20/20 [==============================] - 1s 28ms/step - loss: 1.8455 - accuracy: 0.4978 - val_loss: 2.1177 - val_accuracy: 0.4944
Epoch 9/100
20/20 

c:\Users\Administrator\anaconda3\envs\stock\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Administrator\anaconda3\envs\stock\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Administrator\anaconda3\envs\stock\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

In [11]:
test_answer = pd.read_csv('test_answer.csv', encoding = 'cp949')
X_test = test_answer['exp']
y_test = test_answer['integrated_category']
X_test_seq = tokenizer.texts_to_sequences(X_test)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_seq_length)

pred = model_cnn.predict(X_test_pad)
pred_new = pred.argmax(axis=1)

predicted_categories2 = label_encoder.inverse_transform(pred_new)

# Calculate accuracy
accuracy = accuracy_score(y_test, predicted_categories2)

# Calculate F1 score
f1 = f1_score(y_test, predicted_categories2, average='weighted')  # You can choose 'macro' or 'micro' if needed

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")

14/14 [==============================] - 0s 3ms/step
Accuracy: 0.7482517482517482
F1 Score: 0.7210677869082457


In [12]:
concatenated_df = pd.concat([df, data], ignore_index=True)
concatenated_df.to_csv('통합.csv', index=False, encoding='cp949')